In [62]:
import requests
from bs4 import BeautifulSoup

base_url="http://pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s="

properties_details= None

for page in range(0,30,10):
    req=requests.get(base_url + str(page) + ".html", headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
    content=req.content

    soup=BeautifulSoup(content,"html.parser")
    
    if (properties_details is None):
        properties_details=soup.find_all("div",{"class":"propertyRow"})
    else:
        properties_details.extend(soup.find_all("div",{"class":"propertyRow"}))

In [65]:
houses=[]
for house in properties_details:
    house_details={}
    house_details["Address"]=house.find_all("span", {"class":"propAddressCollapse"})[0].text
    house_details["locality"]=house.find_all("span", {"class":"propAddressCollapse"})[1].text
    house_details["Price"]=house.find("h4", {"class":"propPrice"}).text.replace("\n","").replace(" ","")
    try:
        house_details["Beds"]=house.find("span",{"class":"infoBed"}).find("b").text
    except:
        house_details["Beds"]=None
    
    try:
        house_details["Area"]=house.find("s pan",{"class":"infoSqFt"}).find("b").text
    except:
        house_details["Area"]=None
    
    try:
        house_details["Full Baths"]=house.find("span",{"class":"infoValueFullBath"}).find("b").text
    except:
        house_details["Full Baths"]=None
    
    try:
        house_details["Half Baths"]=house.find("span",{"class":"infoValueHalfBath"}).find("b").text
    except:
        house_details["Half Baths"]=None
        
    for colum_group in house.find_all("div", {"class":"columnGroup"}):
        
        for feature_group, feature_name in \
        zip(colum_group.find_all("span",{"class":"featureGroup"}),\
            colum_group.find_all("span",{"class":"featureName"})):
            
            if "Lot Size" in feature_group.text:
                house_details["Lot Size"]=feature_name.text
    
    houses.append(house_details)

In [68]:
import pandas
df=pandas.DataFrame(houses)

df.to_csv("Output.csv")